<a href="https://colab.research.google.com/github/jmhuer/utaustin_optimization/blob/main/homework4/homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 4

In this problem set you will get some practice with the proximal gradient algorithm, and also acceleration. Specifically, you will be implementing ISTA and FISTA

In [2]:
!pip install gdown
! git clone https://github.com/jmhuer/optimization_tools
from optimization_tools.utils import download_gdrive

# we pass the ID between two slashes '/{ID}/'  
a = '1YDmMRdgRRJSKaWGR7c0g6mzj73Bzf9MU'
b ='1ZdnrCU7YotTAU0duRHTvUUF-k23lG7Wd'
x = '1c4YLkNr7Us61nD6OIdniDI3wac_-UQXS'
y = '1HYFUuBzHQoePElmLy1JOiB8ZmpxVpYPv'
bg = '1-H2_UgwnBImSdFs0YgbbY0JeOkvlh86f'


download_gdrive(a)
download_gdrive(b)
download_gdrive(x)
download_gdrive(y)
download_gdrive(bg)



Cloning into 'optimization_tools'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 46 (delta 12), reused 39 (delta 8), pack-reused 0
Unpacking objects: 100% (46/46), done.
Downloading...
From: https://drive.google.com/uc?id=1YDmMRdgRRJSKaWGR7c0g6mzj73Bzf9MU
To: /content/A.npy
100%|##########| 354k/354k [00:00<00:00, 22.6MB/s]

Downloading...
From: https://drive.google.com/uc?id=1ZdnrCU7YotTAU0duRHTvUUF-k23lG7Wd
To: /content/b.npy
100%|##########| 3.62k/3.62k [00:00<00:00, 11.7MB/s]

Downloading...
From: https://drive.google.com/uc?id=1c4YLkNr7Us61nD6OIdniDI3wac_-UQXS
To: /content/X.npy
100%|##########| 160k/160k [00:00<00:00, 42.2MB/s]

Downloading...
From: https://drive.google.com/uc?id=1HYFUuBzHQoePElmLy1JOiB8ZmpxVpYPv
To: /content/y.npy
100%|##########| 8.08k/8.08k [00:00<00:00, 17.2MB/s]

Downloading...
From: https://drive.google.com/uc?id=1-H2_UgwnBImSdFs0YgbbY0JeOkvlh86f
To: /con

# Problem 1: Gradient Descent and Acceleration
In this problem you will explore the impact of ill-conditioning on gradient descent, and will then see how acceleration can improve the situation. This exercise will walk you through a very similar situation as to what we saw in the lecture videos that illustrate the performance of gradient descent vs accelerated gradient descent as the condition number (ratio of largest to smallest eigenvalues of the Hessian) increases. This is a ``toy'' problem, but it is still instructive regarding the performance of these two algorithms.

You will work with the following simple function:
$$
f(x) = \frac{1}{2}x^{\top}Qx,
$$
where $Q$ is a 2 by 2 matrix, as defined below.

In [ ]:

class Quadratic:
    def __init__(self, Q, b, c, input_size = 2):
        self.input_size = input_size
        self.Q = Q
        self.b = b
        self.c = [c]
    def eval(self, x):
        return .5 * np.dot(x.T, np.dot(self.Q, x))
    def gradient(self,x):
        return np.dot(self.Q, x)
    def b_smooth(self):
        return la.eigvals(self.Q).min()
    def a_sconvex(self):
        return la.eigvals(self.Q).max()



In [ ]:
# We create the data for this simple problem. We will create three quadratics.
# Q_wc -- this is a well-conditioned matrix
# Q_ic -- this is an ill-conditioned matrix
# Q_sic -- this is... a somewhat-ill-conditioned matrix (a technical term!)

import numpy as np
Q_wc = np.array([[1,0.3],[0.3,1]]); q = np.array([0,0]);
Q_sic = np.array([[1,0.85],[0.85,1]]); q = np.array([0,0]);
Q_ic = np.array([[1,0.99],[0.99,1]]); q = np.array([0,0]);

import plotly.graph_objects as graph
import math
import numpy.linalg as la
from tqdm import tqdm

class AGD:
    def __init__(self, function, x_init, epochs, learning_rate,  momentum = None):
        if momentum==None: #default
            self.get_theta = lambda theta_past : (1 + np.sqrt(1+ 4*theta_past**2) ) / 2
            self.get_momentum = lambda theta_past : (theta_past - 1)/self.get_theta(theta_past) 
        else: #constant momentum
            self.get_theta = lambda dummy: 0
            self.get_momentum = lambda dummy: momentum
        self.function = function
        self.x_init = x_init
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.history = {"step": [],
                        "function_vals": [],
                        "grad_vals": [], 
                        "x_vals": [self.x_init], ##initialize x
                        "y_vals": [self.x_init],  ##initialize y
                        "theta": [1], ##initialize theta
                        "opt_dif": [],
                        "test_loss": []}
    def go(self, test_set = None):
        for i in tqdm(range(self.epochs)):
          ##RETRIVE OLD VALS
          x_old = self.history['x_vals'][-1]
          y_old = self.history['y_vals'][-1]
          ##EVALUATE HERE
          f = self.function.eval(x_old)
          g = self.function.gradient(x_old)
          m = self.get_momentum(self.history['theta'][-1])
          ##UPDATE HERE : if lr rate is lamda function act different
          y = x_old - self.learning_rate(i) * g
          x = (1 + m)*y - m * y_old
          ##STORE HISTORY
          self.history['step'].append(i)
          self.history['function_vals'].append(float(f))
          self.history['grad_vals'].append(g)
          self.history['x_vals'].append(x)
          self.history['y_vals'].append(y)
          self.history['theta'].append(self.get_theta(self.history['theta'][-1]))
          if test_set:  
            self.history['test_loss'].append(float(self.function.test_loss(data = test_set, weights = x)))


class GD:
    def __init__(self, function, x_init, epochs, learning_rate):
        self.function = function
        self.x_init = x_init
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.history = {"step": [],
                        "function_vals": [],
                        "grad_vals": [], 
                        "x_vals": [self.x_init], 
                        "opt_dif": [],
                        "test_loss": []}
    def go(self, test_set = None):
        for i in tqdm(range(self.epochs)):
          ##RETRIVE OLD VALS
          x_old = self.history['x_vals'][-1]
          ##EVALUATE HERE
          y = self.function.eval(x_old)
          g = self.function.gradient(x_old)
          ##UPDATE HERE 
          x = x_old - self.learning_rate(i) * g
          ##STORE HISTORY
          self.history['step'].append(i)
          self.history['function_vals'].append(float(y))
          self.history['grad_vals'].append(g)
          self.history['x_vals'].append(x)
          if test_set:  
              self.history['test_loss'].append(float(self.function.test_loss(data = test_set, weights = x)))



## Part (A): 
Consider the quadratic functions $f_{wc}$, $f_{sic}$, and $f_{ic}$ defined by the quadratic matrices above. For each of these, say whether they are $\beta$-smooth and/or $\alpha$-strongly convex, and if so, compute the value of the condition number, $\kappa = \beta/\alpha$ for each function.

In [ ]:

functions = [Quadratic(Q_wc, b=np.array([0,3]), c=0),
             Quadratic(Q_sic,b=np.array([0,3]), c=0),
             Quadratic(Q_ic, b=np.array([0,3]), c=0) ]

for f in functions : 
  b = f.b_smooth()
  a = f.a_sconvex()
  cnd = b/a
  print("b-smooth: {} \t a-strongly convex: {} \t condition convex: {}".format(b,a,cnd) )  


b-smooth: 0.7 	 a-strongly convex: 1.2999999999999998 	 condition convex: 0.5384615384615385
b-smooth: 0.15000000000000013 	 a-strongly convex: 1.85 	 condition convex: 0.08108108108108115
b-smooth: 0.010000000000000009 	 a-strongly convex: 1.99 	 condition convex: 0.005025125628140708


## Part (B): 
Compute the best fixed step size for gradient descent, and the best parameters for accelerated gradient descent. For each function, plot the error $(f(x_t) - f(x^{\ast})$ as a function of the number of iterations. For each function, plot these on the same plot so you can compare -- so you should have 3 plots total.

In [ ]:

functions = [Quadratic(Q_wc, b=np.array([0,3]), c=0),
             Quadratic(Q_sic,b=np.array([0,3]), c=0),
             Quadratic(Q_ic, b=np.array([0,3]), c=0) ]


plot_titles  = ['Well condition matrix',
                'somewhat-ill-conditioned matrix',
                'ill condition matrix' ]

for (f, title) in zip(functions, plot_titles):
  a, b = f.b_smooth(), f.a_sconvex()
  k = b/a 
  gda = AGD(f, x_init = np.array([1,5]), epochs=50, learning_rate = lambda x : 1 / b, momentum = (np.sqrt(k) - 1) / (np.sqrt(k) + 1))
  gda2 = AGD(f, x_init = np.array([1,5]), epochs=50, learning_rate = lambda x : 1 / b)
  gdb =  GD(f, x_init = np.array([1,5]), epochs=50, learning_rate = lambda x : 2 / (a + b))

  gda.go()
  gda2.go() 
  gdb.go()
  # plot the data
  all_history = {"AGD: lr = 1/k" : gda.history,
                 "AGD_old_m: lr = 1/k" : gda.history,
                 "GD: lr = 1/k"  : gdb.history}

  fig = graph.Figure(layout=graph.Layout(title=graph.layout.Title(text=title)))
  for i in all_history:
      fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                                  y    = all_history[i]["function_vals"],
                                  name = i))
  fig.show()





100%|██████████| 50/50 [00:00<00:00, 43736.23it/s]


100%|██████████| 50/50 [00:00<00:00, 41072.31it/s]


100%|██████████| 50/50 [00:00<00:00, 23492.24it/s]


# Problem 2: ISTA and FISTA
Recall the least squares problem with $\ell^1$ regularization from the previous homework:
$$
\min_x \left[f(x) = \frac{1}{2}\|{Ax-b}\|_2^2 + \lambda \|{x}\|_1 \right]
$$

Recall key characteristics of this problem: it is nonsmooth due to the regularization term, and it is not strongly convex when $A$ has more columns than rows. This is why you used the sub-gradient method on the previous problem set, rather than Gradient descent. 

Recall the goal of the proximal gradient algorithm: when we have a composite function, i.e., a function of the form $f(x) = g(x) + h(x)$, if $g(x)$ is $\beta$-smooth and $h(x)$ is ``simple'' in the sense that it has a simple prox function, then rather than using the subgradient method, we can get much better results by using proximal gradient, which takes advantage of the fact that $g(x)$ is smooth. We can improve this further by combining the proximal gradient method with acceleration. 

Using the same data (same $A$ and $b$) as in Problem Set 3, minimize $f(x)$ using $10^4$ iterations with $t=0$ and $x_0 =0$. 

Use the proximal gradient algorithm, also known as ISTA for the case where $f$ is the LASSO objective. Now use the accelerated proximal gradient algorithm, also, known as FISTA. Plot these results on the same plot as your results for sub-gradient descent from the previous lecture. 

In [ ]:
import numpy as np
import numpy.random as rn
import numpy.linalg as la
import matplotlib.pyplot as plt
import time

class LASSO:
    def __init__(self, A, b, lmda = 1e-3):
        self.A = A
        self.b = b
        self.lmda = lmda
        self.loss = lambda weight : (1./b.shape[0])*(.5)*np.sum((np.dot(A,weight)-b)**2)
        self.reg  = lambda weight : self.lmda*la.norm(weight,1)
        self.grad_loss = lambda weight : (1./b.shape[0])*np.dot(A.T,np.dot(A,weight)-b)
        self.grad_reg  = lambda weight : self.lmda*la.norm(weight,1) 
        self.optimal = 0
    def eval(self, weight):
        return self.loss(weight) + self.reg(weight)
    def gradient(self,weight):
        return self.subgradient(weight)      
    def subgradient(self,weight):
        return self.grad_loss(weight) + self.grad_reg(weight)
    def loss_gradient(self,weight):
        return self.grad_loss(weight)      

In [ ]:

class ISTA:
    def __init__(self, function, x_init, epochs, learning_rate, optimal=None):
        self.function = function
        self.x_init = x_init
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.optimal = optimal
        self.history = {"step": [],
                        "function_vals": [],
                        "grad_vals": [], 
                        "x_vals": [self.x_init], 
                        "opt_dif": [],
                        "test_loss": []}
    def prox_operator(self, z, alpha):
        return np.sign(z)*np.maximum(abs(z)-alpha,0)
    def go(self, sumarize=True):
        for i in range(self.epochs):
          ##RETRIVE OLD VALS
          x_old = self.history['x_vals'][-1]
          ##EVALUATE HERE
          f = self.function.eval(x_old)
          g = self.function.loss_gradient(x_old)
          ##UPDATE HERE : if lr rate is lamda function act different
          x = self.prox_operator(x_old - self.learning_rate(i) * g, self.learning_rate(i)*self.function.lmda )
          ##STORE HISTORY
          self.history['step'].append(i)
          self.history['function_vals'].append(float(f))
          self.history['grad_vals'].append(g)
          self.history['x_vals'].append(x)



class FISTA:
    def __init__(self, function, x_init, epochs, learning_rate, optimal=None, momentum=None):
        self.function = function
        self.x_init = x_init
        if momentum==None: #default
          self.get_theta = lambda theta_past : (1 + np.sqrt(1+ 4*theta_past**2) ) / 2
          self.get_momentum = lambda theta_past : (theta_past - 1)/self.get_theta(theta_past) 
        else: #constant momentum
          self.get_theta = lambda dummy: 0
          self.get_momentum = lambda dummy: momentum
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.optimal = optimal
        self.history = {"step": [],
                        "function_vals": [],
                        "grad_vals": [], 
                        "x_vals": [self.x_init], ##initialize x
                        "y_vals": [self.x_init],  ##initialize y
                        "theta": [1], ##initialize theta
                        "opt_dif": [],
                        "test_loss": []}
    def prox_operator(self, z, alpha):
        return np.sign(z)*np.maximum(abs(z)-alpha,0)
    def go(self, sumarize=True):
        for i in range(self.epochs):
          ##RETRIVE OLD VALS
          x_old = self.history['x_vals'][-1]
          y_old = self.history['y_vals'][-1]
          theta_old = self.history['theta'][-1]
          ##EVALUATE HERE
          f = self.function.eval(x_old)
          g = self.function.loss_gradient(x_old)
          ##UPDATE HERE : if lr rate is lamda function act different
          m = self.get_momentum(theta_old)
          y = self.prox_operator(x_old - self.learning_rate(i) * g, self.learning_rate(i)*self.function.lmda)
          x = (1 + m)*y - m * y_old
          ##STORE HISTORY
          self.history['step'].append(i)
          self.history['function_vals'].append(float(f))
          self.history['grad_vals'].append(g)
          self.history['x_vals'].append(x)
          self.history['y_vals'].append(y)
          self.history['theta'].append(self.get_theta(theta_old))



In [ ]:
A = np.load("A.npy")
b = np.load("b.npy")

sm , sc = (1/A.shape[0])*la.norm(A,2)**2,(1/A.shape[0])*la.norm(A,-2)**2

gd    = GD(LASSO(A,b,   lmda=1e-5), x_init = np.random.rand(100)*10, epochs = int(5e2), learning_rate = lambda dummy : 5e-2)
ista  = ISTA(LASSO(A,b, lmda=1e-5), x_init = np.random.rand(100)*10, epochs = int(5e2), learning_rate = lambda dummy : 1/sm )
fista = FISTA(LASSO(A,b,lmda=1e-5), x_init = np.random.rand(100)*10, epochs = int(5e2), learning_rate = lambda dummy : 1/sm  )
gd.go()
ista.go()
fista.go()

# plot the data
all_history = { "GD"  : gd.history ,
               "fista": fista.history,
               "ista" :  ista.history}

fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text="GD vs ISTA vs FISTA")))
for i in all_history:
    fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                                y    = all_history[i]["function_vals"],
                                name = i))
fig.show()





100%|██████████| 500/500 [00:00<00:00, 8734.42it/s]


# Problem 3: Optional -- Why we use LASSO

As an optional exercise, you may want to play around with Lasso and explore its properties in the context of machine learning problems. 

To do this: (A) Generate data for yourself at random: create a $n \times d$ matrix $A$ where each entry comes from a standard Gaussian. Choose $d$ much larger than $n$, say, $d = 1000$ and $n = 100$. Now choose the true solution, $x^{\ast}$ to be a $k$-sparse vector. You can do this in many ways. One simple approach is to let $x^{\ast}$ equal 10 on $k=5$ randomly chosen entries, and then zero every where else. Finally, generate $y$ according to
$$
y = Ax + \epsilon,
$$
where $\epsilon$ is zero mean Gaussian noise with variance $0.1$.

Now solve (via an algorithm of your choice) Lasso. Note that you will have to search for a good value for $\lambda$. Compare the solution you get $\hat{x}_{\rm lasso}$ with the true solution, as you vary $\lambda$. You may also want to compare it to the solution when you do not have any regularization. 


# Problem 4: Logistic Regression

Logistic regression is a simple statistical classification method which models
the conditional distribution of the class variable $y$ being equal to class $c$
given an input $x \in \mathbb{R}^n$. We will examine two classification tasks, one
classifying newsgroup posts, and the other classifying digits. In these tasks
the input $x$ is some description of the sample (e.g., word counts in the news
case) and $y$ is the category the sample belongs to (e.g., sports, politics).
The Logistic Regression model assumes the class distribution conditioned on $x$
is log-linear:
$$
p(y=c|x,b_{1:C}) = \frac{e^{-b_c^\top x}}{\sum_{j=1}^C e^{-b_j^\top x}},
$$
where $C$ is the total number of classes, and the denominator sums over all
classes to ensure that $p(y|x)$ is a proper probability distribution. Each
class $c \in {1,2, \dots, C}$ has a parameter $b_c$, and $\mathbf{b} \in
\mathbb{R}^{nC}$ is the vector of concatenated parameters $\mathbf{b} =
[b_1^\top,b_2^\top,\dots,b_C^\top]^\top$.  Let $X \in \mathbb{R}^{N \times n}$ be the
data matrix where each sample $x_i^\top$ is a row and $N$ is the number of
samples.  The maximum likelihood approach seeks to find the parameter
$\mathbf{b}$ which maximizes the likelihood of the classes given the input data
and the model:

$$
\max_{b_{1:C}} \; p(y|x,b_{1:C}) = \prod_{i=1}^N p(y_i|x_i,b_{1:C}) = \prod_{i=1}^N \frac{e^{-b_{y_i}^\top x_i}}{\sum_{j=1}^C e^{-b_j^\top x_i}}.
$$

For the purposes of optimization, we can equivalently minimize the negative log
likelihood:
$$
\min_\mathbf{\beta} \ell(\mathbf{\beta}) = -\log p(\textbf{y}|X, \mathbf{\beta}) = \sum_{i=1}^N \left( \beta_{y_i}^\top x_i + \log{\sum_{j=1}^C e^{-\beta_j^\top x_i}} \right).
$$

After optimization, the model can be used to classify a new input by choosing
the class that the model predicts as having the highest likelihood; note that
we don't have to compute the normalizing quantity $\sum_{j=1}^C e^{-b_j^\top x}$
as it is constant across all classes:
$$
y = \arg\max_j p(y=j| x, \mathbf{\beta}) = \arg\min_j \beta_j^\top x
$$
In this problem, you will optimize the logistic regression model for the two
classification tasks mentioned above which vary in dimension and number of
classes. The newsgroup dataset that we consider here has $C=20$. 

We will compare the performance of gradient descent and Nesterov's accelerated gradient
method on the $\ell^2$-regularized version of the logistic regression model:
$$
\min_{\boldsymbol{\beta}} = \frac{1}{N} \sum_{i=1}^N \left( \beta_{y_i}^\top x_i + \log{\sum_{j=1}^C e^{-\beta_j^\top x_i}} \right) + \mu \|\boldsymbol{\beta}\|^2.
$$

In this homework, we will use the training and testing data contained in the four csv files in logistic\_news.zip. In a later homework, we will look into the digits dataset (MNIST).



In [ ]:
#sample code to load in logistic_news.zip
#we also create a Z matrix, useful for multiclass logistic regression optimization
import zipfile as zipfile
import csv

class Data:
    def __init__(self,X,Y):
        self.X=X
        self.Y=Y   

def loaddata(filename):
    import io
    data=[]
    with zipfile.ZipFile(filename) as z:
        for filename in z.namelist():
            if filename[0]=='X'or filename[0]=='y':
                each=[]
                with z.open(filename) as csvDataFile:
                    csvReader=csv.reader(csvDataFile)
                    csvDataFile_utf8 = io.TextIOWrapper(csvDataFile, 'utf-8')
                    csvReader=csv.reader(csvDataFile_utf8)
                    for row in csvReader:
                        each.append(row)
                    each==[[float(string) for string in row] for row in each]
                    each=np.asarray(each)
                    data.append(each)    
    X_te=data[0].astype(float)
    X_tr=data[1].astype(float)
    y_te=data[2][0].astype(int)
    y_tr=data[3][0].astype(int)
    Z_tr,Z_te=[],[]
    for j in range(len(np.unique(y_tr))):
        Z_tr.append(np.sum(X_tr[np.where(y_tr==j)[0],:],axis=0))
        Z_te.append(np.sum(X_te[np.where(y_te==j)[0],:],axis=0))
    Z_tr=np.asarray(Z_tr).T
    Z_te=np.asarray(Z_te).T
    train= Data(X_tr,y_tr)
    train.Z=Z_tr
    test = Data(X_te,y_te)
    test.Z=Z_te
    return train,test

train,test=loaddata('./logistic_news.zip')

## Part (A) -- Optional -- 
Find the value of $\mu$ that gives you (approximately) the best generalization performance (error on test set). You obtain this by solving the the above optimization problem for different values of $\mu$, and then checking the performance of the solution on the testing set, using the unregularized logistic regression loss. Note that this is not a question about an optimization method.

What value do you get for the test loss after convergence?  

## Part (B) 

If you did Part (A), use the value of $\mu$ you found there. If you did not, use $\mu = 0.001$. 

Plot the loss against iterations for both the test and training data
using the value of $\mu$ from part (a).

In [ ]:


class Logistic_loss:
    def __init__(self, data, init_weights, mou=0.001):
        self.X = data.X
        self.Y = data.Y
        self.Z = data.Z
        self.N = np.shape(self.X)
        self.mou = mou
    def eval(self, weights):
        X,Y,Z=self.X,self.Y,self.Z
        loss=(1./Y.shape[0])*(np.trace(np.dot(weights.T,Z))+np.sum(np.log(np.sum(np.exp(-np.dot(X,weights)),axis=1))))
        reg=self.mou*la.norm(weights,'fro')**2
        return loss + reg
    def gradient(self,weights):
        return self.subgradient(weights)
    def subgradient(self,weights):
        n,d= self.N
        X,Y,Z=self.X,self.Y,self.Z
        w=(np.exp(-np.dot(X,weights)).T/np.sum(np.exp(-np.dot(X,weights)),axis=1)).T
        g_loss=(1./Y.shape[0])*(Z+np.dot(-X.T,w))
        g_reg=2*self.mou*weights
        return g_loss + g_reg
    def test_loss(self, data, weights):
        X,Y,Z=data.X,data.Y,data.Z
        loss=(1./Y.shape[0])*(np.trace(np.dot(weights.T,Z))+np.sum(np.log(np.sum(np.exp(-np.dot(X,weights)),axis=1))))
        reg=self.mou*la.norm(weights,'fro')**2
        return loss # no reg term for testing 
            
            



## Part (C) 

How do the two algorithms differ in performance, and how does this change
as you decrease $\mu$? 

In [ ]:
model = Logistic_loss(data=train, init_weights= np.zeros(train.Z.shape))

gd = AGD(model, x_init= np.zeros(train.Z.shape), epochs=int(1e2), learning_rate= lambda s : 1e-1)
gd2 = GD(model, x_init= np.zeros(train.Z.shape) , epochs=int(1e2), learning_rate= lambda s : 1e-1)
gd.go(test_set=test)
gd2.go(test_set=test)

all_history = {"AGD":gd.history, "GD": gd2.history}

## we plot train loss
fig = graph.Figure(layout=graph.Layout(title=graph.layout.Title(text= "logistic loss function value vs steps")))
for i in all_history:
  fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                              y    = all_history[i]["function_vals"],
                              name = " " + i))
fig.show()



## we plot test loss
fig = graph.Figure(layout=graph.Layout(title=graph.layout.Title(text="test logistic loss function value vs steps")))
for i in all_history:
  fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                              y    = all_history[i]["test_loss"],
                              name = " " + i))
fig.show()




100%|██████████| 100/100 [00:49<00:00,  2.01it/s]


In [ ]:
tqdm._instances.clear()
